In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from ragelo.utils import load_answers_from_multiple_csvs
import glob
import os
from getpass import getpass
import openai

In [24]:
data_folder = "../data/"
csvs = glob.glob(f"{data_folder}rag_response_*.csv")

In [25]:
if not (openai_api_key := os.environ.get("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

RAGelo is completely independent from your retrieval pipeline. All that it needs are, for each agent/pipeline, their answers and the documents retrieved when building the answers.

In [26]:
queries = load_answers_from_multiple_csvs(csvs, query_text_col="question")
query_ids = {q.query: q.qid for q in queries}
query_dict = {q.qid: q for q in queries}

In [27]:
import pandas as pd
def parse_docs(raw_docs) -> list[tuple[str, str]]:
    docs = raw_docs.split("\n")
    documents = []
    for d in docs:
        doc_text = d.split("document:", maxsplit=1)[1]
        doc_source = d.split("source:", maxsplit=1)[1]
        documents.append((doc_source, doc_text))
    return documents

for csv in csvs:
    df = pd.read_csv(csv)
    for i, row in df.iterrows():
        query_id = query_ids[row["question"]]
        answer = row["answer"]
        docs = parse_docs(row["contexts"])
        query = query_dict[query_id]
        for doc_source, doc_text in docs:
            query.add_retrieved_doc(doc_text, doc_source)


## Evaluate retrieved documents

In [28]:
from ragelo import (
    Query,
    get_answer_evaluator,
    get_llm_provider,
    get_retrieval_evaluator,
)
from ragelo.types.configurations import DomainExpertEvaluatorConfig

llm_provider = get_llm_provider("openai", model_name="gpt-4o", max_tokens=2048)

In [29]:
retrieval_evaluator_config = DomainExpertEvaluatorConfig(
    expert_in="the details of how to better use the Qdrant vector database and vector search engine",
    rich_print=True,
)
retrieval_evaluator = get_retrieval_evaluator(llm_provider=llm_provider, config=retrieval_evaluator_config)
queries = retrieval_evaluator.batch_evaluate(queries)

ValueError: Unknown retrieval evaluator domain_expert
Valid options are []

In [19]:
queries

<coroutine object BaseRetrievalEvaluator.batch_evaluate at 0x13a808040>

In [19]:
len(eval(df['contexts'][0]))

1

In [75]:
from typing import Optional
from pathlib import Path
import csv
from ragelo import AgentAnswer
from ragelo.utils import load_answers_from_multiple_csvs



In [38]:
import csv
with open(csvs[0]) as f:
    reader = csv.DictReader(f)
    for l in reader:
        break
    # columns = reader.fieldnames

In [40]:
l.keys()

dict_keys(['question', 'answer', 'contexts', 'ground_truth'])

In [33]:
colu

['question', 'answer', 'contexts', 'ground_truth']

In [29]:
q[1]['question']

'What is vaccum optimizer ?'